In [1]:
import $ivy.`io.github.siddhartha-gadgil::provingground-core-jvm:0.1.1-SNAPSHOT`

import $ivy.$                                                                   

## Symbolic Algebra for natural numbers

To efficiently manipulate expressions in natural numbers, or more generally rings (and fields), proving-ground has special HoTT types wrapping scala types that are Rings, Rigs, Fields etc in the _spire_ library. As a consequence:

* Symbolic expressions that are equal become _definitionally_ equal, i.e., equal as scala objects.
* We define recursion which expands for (sums with) literals
* Expressions involving literals and variables are simplified as much as possible.


The ring of natural numbers is an object NatRing. This has

* a HoTT type _NatTyp_, 
* a scala type _Nat_
* a scala representation
* a (spire) ring structure on the underlying terms.

In [2]:
import provingground._, induction._, scalahott._
import NatRing._

import provingground._, induction._, scalahott._

import NatRing._

In [3]:
val n = "n" :: NatTyp
val m = "m" :: NatTyp
val k = "k" :: NatTyp

n: RepTerm[spire.math.SafeLong] = RepSymbObj(Name("n"), Nat.Typ)
m: RepTerm[spire.math.SafeLong] = RepSymbObj(Name("m"), Nat.Typ)
k: RepTerm[spire.math.SafeLong] = RepSymbObj(Name("k"), Nat.Typ)

Spire implicits let us use the addition and multiplication operations.

In [4]:
import spire.math._
import spire.algebra._
import spire.implicits._

import spire.math._

import spire.algebra._

import spire.implicits._

### Addition and multiplication

A sum gives a SigmaTerm, which only stores  a set of terms being added.

In [5]:
n + m

res4: LocalTerm = SigmaTerm(
  Set(RepSymbObj(Name("n"), Nat.Typ), RepSymbObj(Name("m"), Nat.Typ))
)

In [6]:
(n + m) + n

res5: LocalTerm = SigmaTerm(
  Set(
    RepSymbObj(Name("m"), Nat.Typ),
    RepSymbObj(
      ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj(Name("n"), Nat.Typ)),
      Nat.Typ
    )
  )
)

Addition is commutative and associative, even when it involves repeated terms.

In [7]:
n + m == m + n
(n + m) + k == n + (m + k)

res6_0: Boolean = true
res6_1: Boolean = true

In [8]:
(n + n) + m == (n + m) + n

res7: Boolean = true

Similarly, multiplication is commutative and associative, and  distributes over addition. Multiplication gives Pi-terms with parameter a map to exponents.

In [9]:
n * m == m * n

res8: Boolean = true

In [10]:
n * (m * k)

res9: LocalTerm = PiTerm(
  Map(
    RepSymbObj(Name("k"), Nat.Typ) -> 1,
    RepSymbObj(Name("n"), Nat.Typ) -> 1,
    RepSymbObj(Name("m"), Nat.Typ) -> 1
  )
)

In [11]:
(n * m) * k

res10: LocalTerm = PiTerm(
  Map(
    RepSymbObj(Name("k"), Nat.Typ) -> 1,
    RepSymbObj(Name("n"), Nat.Typ) -> 1,
    RepSymbObj(Name("m"), Nat.Typ) -> 1
  )
)

In [12]:
n * (m + k)

res11: LocalTerm = SigmaTerm(
  Set(
    PiTerm(
      Map(
        RepSymbObj(Name("n"), Nat.Typ) -> 1,
        RepSymbObj(Name("m"), Nat.Typ) -> 1
      )
    ),
    PiTerm(
      Map(
        RepSymbObj(Name("k"), Nat.Typ) -> 1,
        RepSymbObj(Name("n"), Nat.Typ) -> 1
      )
    )
  )
)

In [13]:
n *(m + k) == (n * m) + (n * k)

res12: Boolean = true

In [14]:
n + 1

res13: LocalTerm = RepSymbObj(
  ApplnSym(AddLiteral(SafeLongLong(1L)), RepSymbObj(Name("n"), Nat.Typ)),
  Nat.Typ
)

In [15]:
1 + n

res14: RepTerm[SafeLong] = RepSymbObj(
  ApplnSym(AddLiteral(SafeLongLong(1L)), RepSymbObj(Name("n"), Nat.Typ)),
  Nat.Typ
)

In [16]:
(1 + n) + 2

res15: LocalTerm = RepSymbObj(
  ApplnSym(AddLiteral(SafeLongLong(3L)), RepSymbObj(Name("n"), Nat.Typ)),
  Nat.Typ
)

In [17]:
n * n

res16: LocalTerm = PiTerm(Map(RepSymbObj(Name("n"), Nat.Typ) -> 2))

### Symbolic definitions

We can use the expressions from these functions in lambdas. For this we need correct substitution.

In [18]:
import HoTT._
val fn = lmbda(n)(n * n)

import HoTT._

fn: Func[RepTerm[SafeLong], LocalTerm] = LambdaFixed(
  RepSymbObj(n, Nat.Typ),
  PiTerm(Map(RepSymbObj(n, Nat.Typ) -> 2))
)

In [19]:
fn(3)

res18: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(9L)), Nat.Typ)

In [20]:
fn(k)

res19: LocalTerm = PiTerm(Map(RepSymbObj(Name("k"), Nat.Typ) -> 2))

### Recursive definitions

We can define a function f recursively on natural numbers, given the value f(0) and given f(n+1) as a (curryed) function of (n+1) and f(n). This expands  for literals.

In [21]:
val m = lmbda(n)(prod(n + 1))
val factorial = Rec(1: Nat, m)

m: Func[RepTerm[SafeLong], Func[LocalTerm, LocalTerm]] = LambdaFixed(
  RepSymbObj(n, Nat.Typ),
  LambdaFixed(
    RepSymbObj($n, Nat.Typ),
    SigmaTerm(
      Set(
        RepSymbObj($n, Nat.Typ),
        PiTerm(Map(RepSymbObj($n, Nat.Typ) -> 1, RepSymbObj(n, Nat.Typ) -> 1))
      )
    )
  )
)
factorial: Rec[LocalTerm] = Rec(
  RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
  LambdaFixed(
    RepSymbObj(n, Nat.Typ),
    LambdaFixed(
      RepSymbObj($n, Nat.Typ),
      SigmaTerm(
        Set(
          RepSymbObj($n, Nat.Typ),
          PiTerm(Map(RepSymbObj($n, Nat.Typ) -> 1, RepSymbObj(n, Nat.Typ) -> 1))
        )
      )
    )
  )
)

In [22]:
factorial(3)

res21: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(6L)), Nat.Typ)

In [23]:
factorial(5)

res22: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(120L)), Nat.Typ)

In [24]:
factorial(n)

res23: LocalTerm = RepSymbObj(
  ApplnSym(
    Rec(
      RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
      LambdaFixed(
        RepSymbObj(n, Nat.Typ),
        LambdaFixed(
          RepSymbObj($n, Nat.Typ),
          SigmaTerm(
            Set(
              RepSymbObj($n, Nat.Typ),
              PiTerm(
                Map(RepSymbObj($n, Nat.Typ) -> 1, RepSymbObj(n, Nat.Typ) -> 1)
              )
            )
          )
        )
      )
    ),
    RepSymbObj(Name("n"), Nat.Typ)
  ),
  Nat.Typ
)

In [25]:
val g = lmbda(k)(factorial(k * k))

g: Func[RepTerm[SafeLong], LocalTerm] = LambdaFixed(
  RepSymbObj(k, Nat.Typ),
  RepSymbObj(
    ApplnSym(
      Rec(
        RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
        LambdaFixed(
          RepSymbObj(n, Nat.Typ),
          LambdaFixed(
            RepSymbObj($n, Nat.Typ),
            SigmaTerm(
              Set(
                RepSymbObj($n, Nat.Typ),
                PiTerm(
                  Map(RepSymbObj($n, Nat.Typ) -> 1, RepSymbObj(n, Nat.Typ) -> 1)
                )
              )
            )
          )
        )
      ),
      PiTerm(Map(RepSymbObj(k, Nat.Typ) -> 2))
    ),
    Nat.Typ
  )
)

In [26]:
g(3)

res25: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(362880L)), Nat.Typ)

In [27]:
factorial(9)

res26: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(362880L)), Nat.Typ)

### Simplifying recursive functions

If we apply a recursive function to a sum n+k with k a literal (say k = 2), then the result simplifies as much as possible by expanding tail recursively in the literal.

In [28]:
factorial(n + 1)

res27: LocalTerm = SigmaTerm(
  Set(
    RepSymbObj(
      ApplnSym(
        Rec(
          RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
          LambdaFixed(
            RepSymbObj(n, Nat.Typ),
            LambdaFixed(
              RepSymbObj($n, Nat.Typ),
              SigmaTerm(
                Set(
                  RepSymbObj($n, Nat.Typ),
                  PiTerm(
                    Map(
                      RepSymbObj($n, Nat.Typ) -> 1,
                      RepSymbObj(n, Nat.Typ) -> 1
                    )
                  )
                )
              )
            )
          )
        ),
        RepSymbObj(Name("n"), Nat.Typ)
      ),
      Nat.Typ
    ),
    PiTerm(
      Map(
        RepSymbObj(Name("n"), Nat.Typ) -> 1,
        RepSymbObj(
          ApplnSym(
            Rec(
              RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
              LambdaFixed(
                RepSymbObj(n, Nat.Typ),
                LambdaFixed(
                  RepS

In [29]:
val fn = lmbda(n)(factorial(n + 1))

fn: Func[RepTerm[SafeLong], LocalTerm] = LambdaFixed(
  RepSymbObj(n, Nat.Typ),
  SigmaTerm(
    Set(
      RepSymbObj(
        ApplnSym(
          Rec(
            RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
            LambdaFixed(
              RepSymbObj(n, Nat.Typ),
              LambdaFixed(
                RepSymbObj($n, Nat.Typ),
                SigmaTerm(
                  Set(
                    RepSymbObj($n, Nat.Typ),
                    PiTerm(
                      Map(
                        RepSymbObj($n, Nat.Typ) -> 1,
                        RepSymbObj(n, Nat.Typ) -> 1
                      )
                    )
                  )
                )
              )
            )
          ),
          RepSymbObj(n, Nat.Typ)
        ),
        Nat.Typ
      ),
      PiTerm(
        Map(
          RepSymbObj(n, Nat.Typ) -> 1,
          RepSymbObj(
            ApplnSym(
              Rec(
                RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
   

In [30]:
fn(1)

res29: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ)

In [31]:
fn(4)

res30: LocalTerm = RepSymbObj(ScalaSymbol(SafeLongLong(120L)), Nat.Typ)

In [32]:
(n + 2) * (n + 1)

res31: LocalTerm = RepSymbObj(
  ApplnSym(
    AddLiteral(SafeLongLong(2L)),
    SigmaTerm(
      Set(
        PiTerm(Map(RepSymbObj(Name("n"), Nat.Typ) -> 2)),
        RepSymbObj(
          ApplnSym(
            multLiteral(SafeLongLong(3L)),
            RepSymbObj(Name("n"), Nat.Typ)
          ),
          Nat.Typ
        )
      )
    )
  ),
  Nat.Typ
)

In [33]:
(3 * n) * n

res32: LocalTerm = RepSymbObj(
  ApplnSym(
    multLiteral(SafeLongLong(3L)),
    PiTerm(Map(RepSymbObj(Name("n"), Nat.Typ) -> 2))
  ),
  Nat.Typ
)

In [34]:
n * (n * n)

res33: LocalTerm = PiTerm(Map(RepSymbObj(Name("n"), Nat.Typ) -> 3))

In [35]:
(n * k) * k

res34: LocalTerm = PiTerm(
  Map(RepSymbObj(Name("k"), Nat.Typ) -> 2, RepSymbObj(Name("n"), Nat.Typ) -> 1)
)

In [36]:
k * (n * k)

res35: LocalTerm = PiTerm(
  Map(RepSymbObj(Name("k"), Nat.Typ) -> 2, RepSymbObj(Name("n"), Nat.Typ) -> 1)
)

In [37]:
(n * n) * n

res36: LocalTerm = PiTerm(Map(RepSymbObj(Name("n"), Nat.Typ) -> 3))

In [38]:
factorial(n + 2)

res37: LocalTerm = SigmaTerm(
  Set(
    RepSymbObj(
      ApplnSym(
        multLiteral(SafeLongLong(2L)),
        RepSymbObj(
          ApplnSym(
            Rec(
              RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
              LambdaFixed(
                RepSymbObj(n, Nat.Typ),
                LambdaFixed(
                  RepSymbObj($n, Nat.Typ),
                  SigmaTerm(
                    Set(
                      RepSymbObj($n, Nat.Typ),
                      PiTerm(
                        Map(
                          RepSymbObj($n, Nat.Typ) -> 1,
                          RepSymbObj(n, Nat.Typ) -> 1
                        )
                      )
                    )
                  )
                )
              )
            ),
            RepSymbObj(Name("n"), Nat.Typ)
          ),
          Nat.Typ
        )
      ),
      Nat.Typ
    ),
    PiTerm(
      Map(
        RepSymbObj(Name("n"), Nat.Typ) -> 2,
        RepSymbObj(
          ApplnSy

**Recursive expansion:** We see an example of expansion as much as possible.

In [39]:
val func = lmbda(n)(factorial(n+ 2))
func(3)

func: Func[RepTerm[SafeLong], LocalTerm] = LambdaFixed(
  RepSymbObj(n, Nat.Typ),
  SigmaTerm(
    Set(
      PiTerm(
        Map(
          RepSymbObj(
            ApplnSym(
              Rec(
                RepSymbObj(ScalaSymbol(SafeLongLong(1L)), Nat.Typ),
                LambdaFixed(
                  RepSymbObj(n, Nat.Typ),
                  LambdaFixed(
                    RepSymbObj($n, Nat.Typ),
                    SigmaTerm(
                      Set(
                        RepSymbObj($n, Nat.Typ),
                        PiTerm(
                          Map(
                            RepSymbObj($n, Nat.Typ) -> 1,
                            RepSymbObj(n, Nat.Typ) -> 1
                          )
                        )
                      )
                    )
                  )
                )
              ),
              RepSymbObj(n, Nat.Typ)
            ),
            Nat.Typ
          ) -> 1,
          RepSymbObj(n, Nat.Typ) -> 2
        )
      ),
     

In [40]:
func(k) == factorial(k) * (k + 2) * (k + 1)

res39: Boolean = true

In [41]:
1 + 2

res40: Int = 3

In [42]:
findFactor(Literal(2), Literal(4))

res41: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [43]:
findDivisibilty(Literal(2), Literal(4))

res42: Option[AbsPair[Nat, Equality[Nat]]] = Some(
  DepPair(
    RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ),
    Refl(Nat.Typ, RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)),
    LambdaFixed(
      RepSymbObj($c, Nat.Typ),
      IdentityTyp(
        Nat.Typ,
        RepSymbObj(
          ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj($c, Nat.Typ)),
          Nat.Typ
        ),
        RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
      )
    )
  )
)

In [44]:
findDivisibilty(Literal(2), Literal(4)).map(_.typ)

res43: Option[Typ[U] forSome { type U >: x$1 <: Term with Subs[U]; val x$1: AbsPair[Nat, Equality[Nat]] }] = Some(
  SigmaTyp(
    LambdaFixed(
      RepSymbObj($c, Nat.Typ),
      IdentityTyp(
        Nat.Typ,
        RepSymbObj(
          ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj($c, Nat.Typ)),
          Nat.Typ
        ),
        RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
      )
    )
  )
)

In [45]:
findFactor(n *2, n* 4)

res44: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [46]:
findFactor(n * 2, n * 2 * k)

res45: Option[Nat] = Some(RepSymbObj(Name("k"), Nat.Typ))

In [47]:
findFactor(n * k, n * n * k)

res46: Option[Nat] = Some(RepSymbObj(Name("n"), Nat.Typ))

In [48]:
findFactor(n * 2, n * 4 * k)

res47: Option[Nat] = Some(
  RepSymbObj(
    ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj(Name("k"), Nat.Typ)),
    Nat.Typ
  )
)

In [49]:
findFactor(n * 2, n * 7 * k)

res48: Option[Nat] = None

In [50]:
findFactor(n * 2, n * 4 * k) == Some(k * 2)

res49: Boolean = true

In [51]:
findDivisibilty(n * 2, n * 4 * k)

res50: Option[AbsPair[Nat, Equality[Nat]]] = Some(
  DepPair(
    RepSymbObj(
      ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj(Name("k"), Nat.Typ)),
      Nat.Typ
    ),
    Refl(
      Nat.Typ,
      RepSymbObj(
        ApplnSym(
          multLiteral(SafeLongLong(4L)),
          PiTerm(
            Map(
              RepSymbObj(Name("k"), Nat.Typ) -> 1,
              RepSymbObj(Name("n"), Nat.Typ) -> 1
            )
          )
        ),
        Nat.Typ
      )
    ),
    LambdaFixed(
      RepSymbObj($c, Nat.Typ),
      IdentityTyp(
        Nat.Typ,
        RepSymbObj(
          ApplnSym(
            multLiteral(SafeLongLong(2L)),
            PiTerm(
              Map(
                RepSymbObj(Name("n"), Nat.Typ) -> 1,
                RepSymbObj($c, Nat.Typ) -> 1
              )
            )
          ),
          Nat.Typ
        ),
        RepSymbObj(
          ApplnSym(
...

In [52]:
divides(Literal(2))(Literal(3))

res51: SigmaTyp[LocalTerm, Equality[LocalTerm]] = SigmaTyp(
  LambdaFixed(
    RepSymbObj($c, Nat.Typ),
    IdentityTyp(
      Nat.Typ,
      RepSymbObj(
        ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj($c, Nat.Typ)),
        Nat.Typ
      ),
      RepSymbObj(ScalaSymbol(SafeLongLong(3L)), Nat.Typ)
    )
  )
)

In [53]:
findDivisibilty(2, 4)

res52: Option[AbsPair[Nat, Equality[Nat]]] = Some(
  DepPair(
    RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ),
    Refl(Nat.Typ, RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)),
    LambdaFixed(
      RepSymbObj($c, Nat.Typ),
      IdentityTyp(
        Nat.Typ,
        RepSymbObj(
          ApplnSym(multLiteral(SafeLongLong(2L)), RepSymbObj($c, Nat.Typ)),
          Nat.Typ
        ),
        RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
      )
    )
  )
)

In [54]:
findDifference(n+ 4, n + 2)

res53: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [55]:
findDifference(4, 2)

res54: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [56]:
findDifferenceFlip(4, 2)

res55: Option[Nat] = None

In [57]:
findLEQ(2, 4)

res56: Option[DepPair[Nat, Equality[Nat]]] = Some(
  DepPair(
    RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ),
    Refl(Nat.Typ, RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)),
    LambdaFixed(
      RepSymbObj($g, Nat.Typ),
      IdentityTyp(
        Nat.Typ,
        RepSymbObj(
          ApplnSym(AddLiteral(SafeLongLong(2L)), RepSymbObj($g, Nat.Typ)),
          Nat.Typ
        ),
        RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
      )
    )
  )
)

In [58]:
LEQ.unapply(leq(n)(k))

res57: Option[(Nat, Nat)] = Some(
  (RepSymbObj(Name("n"), Nat.Typ), RepSymbObj(Name("k"), Nat.Typ))
)

In [59]:
LEQ.unapply(leq(Literal(2))(Literal(4))) 

res58: Option[(Nat, Nat)] = Some(
  (
    RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ),
    RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
  )
)

In [60]:
val sg = leq(Literal(2))(Literal(4))

sg: SigmaTyp[Nat, Equality[Nat]] = SigmaTyp(
  LambdaFixed(
    RepSymbObj($g, Nat.Typ),
    IdentityTyp(
      Nat.Typ,
      RepSymbObj(
        ApplnSym(AddLiteral(SafeLongLong(2L)), RepSymbObj($g, Nat.Typ)),
        Nat.Typ
      ),
      RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
    )
  )
)

In [61]:
findDifference(n + 2, 2)

res60: Option[Nat] = Some(RepSymbObj(Name("n"), Nat.Typ))

In [62]:
val x = NatTyp.Var

x: RepTerm[SafeLong] = RepSymbObj(Name("$zr"), Nat.Typ)

In [63]:
val eqn = sg.fibers(x).asInstanceOf[IdentityTyp[Nat]]

eqn: IdentityTyp[Nat] = IdentityTyp(
  Nat.Typ,
  RepSymbObj(
    ApplnSym(AddLiteral(SafeLongLong(2L)), RepSymbObj(Name("$zr"), Nat.Typ)),
    Nat.Typ
  ),
  RepSymbObj(ScalaSymbol(SafeLongLong(4L)), Nat.Typ)
)

In [64]:
eqn.dom == NatTyp

res63: Boolean = true

In [65]:
eqn.lhs

res64: Nat = RepSymbObj(
  ApplnSym(AddLiteral(SafeLongLong(2L)), RepSymbObj(Name("$zr"), Nat.Typ)),
  Nat.Typ
)

In [66]:
findDifference(eqn.lhs, x)

res65: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [67]:
x + 2

res66: LocalTerm = RepSymbObj(
  ApplnSym(AddLiteral(SafeLongLong(2L)), RepSymbObj(Name("$zr"), Nat.Typ)),
  Nat.Typ
)

In [68]:
findDifference(x + 2, 2)

res67: Option[Nat] = Some(RepSymbObj(Name("$zr"), Nat.Typ))

In [69]:
eqn.lhs == x + 2

res68: Boolean = true

In [70]:
findDifference(x + 2, x)

res69: Option[Nat] = Some(RepSymbObj(ScalaSymbol(SafeLongLong(2L)), Nat.Typ))

In [71]:
DIV.unapply(divides(n)(k))

res70: Option[(Nat, Nat)] = Some(
  (RepSymbObj(Name("n"), Nat.Typ), RepSymbObj(Name("k"), Nat.Typ))
)

In [72]:
divides(n)(k)

res71: SigmaTyp[LocalTerm, Equality[LocalTerm]] = SigmaTyp(
  LambdaFixed(
    RepSymbObj($c, Nat.Typ),
    IdentityTyp(
      Nat.Typ,
      PiTerm(
        Map(RepSymbObj(Name("n"), Nat.Typ) -> 1, RepSymbObj($c, Nat.Typ) -> 1)
      ),
      RepSymbObj(Name("k"), Nat.Typ)
    )
  )
)

In [73]:
findFactor(n * k, n)

res72: Option[Nat] = None

In [74]:
findFactor(k, n * k)

res73: Option[Nat] = Some(RepSymbObj(Name("n"), Nat.Typ))

In [75]:
DIV.unapply(divides(n)(Literal(7)))

res74: Option[(Nat, Nat)] = Some(
  (
    RepSymbObj(Name("n"), Nat.Typ),
    RepSymbObj(ScalaSymbol(SafeLongLong(7L)), Nat.Typ)
  )
)